In [ ]:
'''
Reconstruction helical data using parallel conebeam rebinning
'''

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

import ct_projector.projector.numpy as ct_projector
import ct_projector.projector.numpy.helical_equiangular_parallel_rebin as ct_helical

import importlib
importlib.reload(ct_helical)

In [ ]:
input_filename = '54_1.mat'

with h5py.File(input_filename, 'r') as f:
    # if a view is valid
    view_valid_a = np.copy(f['sh']['Lookup']['DetA']).flatten()
    view_valid_b = np.copy(f['sh']['Lookup']['DetB']).flatten()
    
    # z position of the source for each view, convert to mm
    zpos_a = np.copy(f['posA']).flatten() / 1000
    zpos_b = np.copy(f['posB']).flatten() / 1000
    
    # angle of the source for each view, convert to radius
    angles_a = np.copy(f['angleA']).flatten() / 180 * np.pi
    angles_b = np.copy(f['angleB']).flatten() / 180 * np.pi
    
    # projection, convert to attenuation
    prjs_a = np.copy(np.copy(f['projA'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5
    prjs_b = np.copy(np.copy(f['projB'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5

In [ ]:
projector = ct_projector.ct_projector()
projector.nv = prjs_a.shape[1]
projector.dso = 595
projector.du = 0.067864004196156 * np.pi / 180 * projector.dsd
projector.dv = 1.0947
projector.off_u = -1.25
projector.rotview = 1152
projector.nx = 640
projector.ny = 640
projector.dx = 0.75
projector.dy = 0.75
projector.dz = 0.75
dtheta = np.pi * 2 / projector.rotview

projector.nu = prjs_a.shape[2]
projector_rebin_a, prjs_rebin_a, angles_a, zrot_a, nview_margin_pre_a, nview_margin_post_a = \
    ct_helical.rebin_helical_to_parallel(projector, prjs_a, angles_a, zpos_a)

projector.nu = prjs_b.shape[2]
projector_rebin_b, prjs_rebin_b, angles_b, zrot_b, nview_margin_pre_b, nview_margin_post_b = \
    ct_helical.rebin_helical_to_parallel(projector, prjs_b, angles_b, zpos_b)

In [ ]:
istart_origin_a = np.where(view_valid_a > 0.5)[0][0]
istart_origin_b = np.where(view_valid_b > 0.5)[0][0]
first_angle_offset = istart_origin_b - istart_origin_a
print(first_angle_offset)

In [ ]:
prjs_rebin_ab, istart_a, istart_b = ct_helical.pad_dual_source_ct_rebinned_projection(
    projector_rebin_a,
    projector_rebin_b,
    prjs_rebin_a,
    prjs_rebin_b,
    angles_a,
    angles_b,
    zrot_a,
    zrot_b,
    first_angle_offset,
    nview_margin_pre_a,
    nview_margin_pre_b,
    nview_margin_post_a,
    nview_margin_post_b,
)

In [ ]:
recon_z_start = zpos_a[istart_a]
recon_z_end = zpos_a[istart_a] + zrot_a * prjs_rebin_ab.shape[1] /  projector.rotview

In [ ]:
ct_projector.set_device(1)
img_a = ct_helical.fbp_long(
    projector_rebin_a,
    prjs_rebin_ab[[0]],
    angles_a[istart_a],
    zpos_a[istart_a],
    zrot_a,
    recon_z_start=recon_z_start,
    recon_z_end=recon_z_end
)

In [ ]:
img_b = ct_helical.fbp_long(
    projector_rebin_b,
    prjs_rebin_ab[[1]],
    angles_b[istart_b],
    zpos_b[istart_b],
    zrot_b,
    recon_z_start=recon_z_start,
    recon_z_end=recon_z_end
)

In [ ]:
plt.figure(figsize=[16, 8])
plt.subplot(121)
plt.imshow(img_a[..., ::-1][0, :, 320, :], 'gray', vmin=0.01596, vmax=0.02356)
plt.subplot(122)
plt.imshow(img_b[..., ::-1][0, :, 320, :], 'gray', vmin=0.01596, vmax=0.02356)

In [ ]:
diff = (img_a - img_b) / 0.019 * 1000
plt.figure(figsize=[8, 8])
plt.imshow(diff[..., ::-1][0, 145, 64:-64, 64:-64], 'gray', vmin=0, vmax=250)

In [ ]:
plt.figure(figsize=[8, 8])
plt.imshow(diff[..., ::-1][0, :, 300, :], 'gray', vmin=0, vmax=250)